In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Github/SindyAutoencoders_project

/content/drive/My Drive/Colab Notebooks/Github/SindyAutoencoders_project


In [3]:
import os
import datetime
import pandas as pd
import numpy as np
from robi.example_LV2 import get_lofka_volterra_data
from src.sindy_utils import library_size
from src.training import train_network
import tensorflow as tf

# Generate data

In [4]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lofka_volterra_data(128, noise_strength=noise_strength)
validation_data = get_lofka_volterra_data(20, noise_strength=noise_strength)

In [ ]:
training_data

# Set up model and training parameters

In [5]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 2
params['model_order'] = 1
params['poly_order'] = 2
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 1e-4
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [7]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'LV2_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    # tf.reset_default_graph() didn't work anymore
    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.04867162927985191, (0.041144263, 73.814285, 1.3691305, 0.90251803)
   validation loss 0.09921310096979141, (0.0895632, 93.44309, 2.9657373, 0.90251803)
decoder loss ratio: 0.326918, decoder SINDy loss  ratio: 1.255512
Epoch 100
   training loss 2.8853635740233585e-05, (1.2610619e-05, 0.018196467, 0.0042847674, 1.3994894)
   validation loss 3.357855530339293e-05, (1.4479771e-05, 0.0365249, 0.01451401, 1.3994894)
decoder loss ratio: 0.000053, decoder SINDy loss  ratio: 0.006144
Epoch 200
   training loss 2.9100610845489427e-05, (1.7294087e-05, 0.009174155, 0.0020424456, 1.0684861)
   validation loss 3.59214172931388e-05, (2.2885373e-05, 0.016070202, 0.007441638, 1.0684861)
decoder loss ratio: 0.000084, decoder SINDy loss  ratio: 0.003150
Epoch 300
   training loss 1.8412702047498897e-05, (8.575811e-06, 0.0076433006, 0.0014312848, 0.8929434)
   validation loss 2.6326953957322985e-05, (1.5568641e-05, 0.013579152, 0.004709635, 0.8929434)
deco

<ipython-input-7-e378c3c436ee>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({**results_dict, **params}, ignore_index=True)
